In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from src.model import Model
from src.layer import binary_sigmoid_unit
import pickle
from sklearn.compose import ColumnTransformer

In [40]:
# waterPump = pd.read_csv('data/waterPump.csv')

# display(waterPump)

In [41]:
# Diccionario para el mapeo
# target_mapping = {
#     (0, 0, 1): 'NF',  # No funcional
#     (0, 1, 0): 'FR',  # Funcional necesitando reparación
#     (1, 0, 0): 'F',  # Funcional
# }

# # Función para aplicar el mapeo usando el diccionario
# def map_target(row):
#     return target_mapping[(row['target_functional'], row['target_functional needs repair'], row['target_non functional'])]

# waterPump = waterPump.copy()
# waterPump['target'] = waterPump.apply(map_target, axis=1)
# waterPump = waterPump.drop(['target_functional', 'target_functional needs repair', 'target_non functional'], axis=1)

# display(waterPump)

# pickle.dump(waterPump, open('data/waterPump.pkl', 'wb'))

In [42]:
# Picke store 
waterPump = pickle.load(open('data/waterPump.pkl', 'rb'))
waterPump 

,amount_tsh,gps_height,longitude,latitude,num_private,population,funder_0,funder_A/co Germany,funder_Aar,funder_Abas Ka,...,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other,target
0,6000.0,1390,34.938093,-9.856322,0,109,0,0,0,0,...,0,0,0,0,1,0,0,0,0,F
1,0.0,1399,34.698766,-2.147466,0,280,0,0,0,0,...,0,0,0,0,1,0,0,0,0,F
2,25.0,686,37.460664,-3.821329,0,250,0,0,0,0,...,0,0,0,0,1,0,0,0,0,F
3,0.0,263,38.486161,-11.155298,0,58,0,0,0,0,...,0,0,0,0,1,0,0,0,0,NF
4,0.0,0,31.130847,-1.825359,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,10.0,1210,37.169807,-3.253847,0,125,0,0,0,0,...,0,0,0,0,1,0,0,0,0,F
59396,4700.0,1212,35.249991,-9.070629,0,56,0,0,0,0,...,0,0,0,0,1,0,0,0,0,F
59397,0.0,0,34.017087,-8.750434,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,F
59398,0.0,0,35.861315,-6.378573,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,F


In [56]:
# Mirar si hay null

waterPump.isnull().sum() == 0

amount_tsh                               True
gps_height                               True
longitude                                True
latitude                                 True
num_private                              True
                                         ... 
waterpoint_type_group_dam                True
waterpoint_type_group_hand pump          True
waterpoint_type_group_improved spring    True
waterpoint_type_group_other              True
target                                   True
Length: 9215, dtype: bool

In [43]:
def categorize_variables(df):

    categorical = []
    numerical = []

    for column in df.columns:
        unique_values = df[column].unique()
        n_unique = len(unique_values)

        if n_unique <= 10:
            categorical.append((column, unique_values.tolist()))
        else:
            numerical.append(column)

    return {
        'categorical': categorical,
        'numerical': numerical
    }

In [45]:
# Hay alguno en categorical que no sea binario?
[ elem for elem in categorize_variables(waterPump)['categorical'] if elem[1] != [0, 1] and elem[1] != [1, 0] ]

[('recorded_by_GeoData Consultants Ltd', [1]), ('target', ['F', 'NF', 'FR'])]

In [46]:
waterPump = waterPump.drop('recorded_by_GeoData Consultants Ltd', axis=1) # Eliminamos columna inncesaria

In [47]:
variables_numericas = categorize_variables(waterPump)['numerical']
variables_numericas

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'population']

In [48]:
waterPump['target'].value_counts()

target
F     32259
NF    22824
FR     4317
Name: count, dtype: int64

In [62]:
XwaterPump = waterPump.drop(['target'], axis=1)
ywaterPump = waterPump['target']

XwaterPump_train, XwaterPump_test, ywaterPump_train, ywaterPump_test = train_test_split(
    waterPump.drop(['target'], axis=1), 
    waterPump['target'], test_size=0.2, 
    random_state=42)

In [63]:
scaler = StandardScaler()

scaler = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), variables_numericas)
    ],
    remainder='passthrough'  # No escalar las demás variables
)

# Ajustar y transformar el conjunto de entrenamiento
XwaterPump_train_scaled = scaler.fit_transform(XwaterPump_train)

# Transformar el conjunto de prueba
XwaterPump_test_scaled = scaler.transform(XwaterPump_test)

XwaterPump_train_scaled = pd.DataFrame(XwaterPump_train_scaled, columns=XwaterPump_train.columns)
XwaterPump_test_scaled = pd.DataFrame(XwaterPump_test_scaled, columns=XwaterPump_test.columns)


In [67]:
# --> One versus rest: NF vs resto

# No funcional vs el resto
ywaterPump_train_nf = ywaterPump_train.copy()
ywaterPump_train_nf = ywaterPump_train_nf.apply(lambda x: 1 if x == 'NF' else 0)

ywaterPump_test_nf = ywaterPump_test.copy()
ywaterPump_test_nf = ywaterPump_test_nf.apply(lambda x: 1 if x == 'NF' else 0)

In [68]:
modelo = Model(n_inputs=XwaterPump_train_scaled.shape[1])

modelo.fit(
    X_train=XwaterPump_train_scaled,
    y_train=ywaterPump_train_nf
    )

Epoch 1/100
1188/1188 [==============================] - 11s 9ms/step - loss: 0.5329 - binary_accuracy: 0.7614 - val_loss: 0.5223 - val_binary_accuracy: 0.7547
Epoch 2/100
1188/1188 [==============================] - 7s 6ms/step - loss: 0.5104 - binary_accuracy: 0.7621 - val_loss: 0.5167 - val_binary_accuracy: 0.7542
Epoch 3/100
1188/1188 [==============================] - 8s 7ms/step - loss: 0.5088 - binary_accuracy: 0.7622 - val_loss: 0.5270 - val_binary_accuracy: 0.7547
Epoch 4/100
1188/1188 [==============================] - 10s 8ms/step - loss: 0.5091 - binary_accuracy: 0.7627 - val_loss: 0.5287 - val_binary_accuracy: 0.7544
Epoch 5/100
1188/1188 [==============================] - 10s 8ms/step - loss: 0.5092 - binary_accuracy: 0.7612 - val_loss: 0.5297 - val_binary_accuracy: 0.7533
Epoch 6/100
1188/1188 [==============================] - 10s 8ms/step - loss: 0.5089 - binary_accuracy: 0.7620 - val_loss: 0.5193 - val_binary_accuracy: 0.7509
Epoch 7/100
1188/1188 [===================

In [69]:
modelo.predict(XwaterPump_test_scaled)

372/372 [==============================] - 1s 3ms/step


array([[0.9971158 ],
       [0.48835418],
       [0.26059082],
       ...,
       [0.97973865],
       [0.33462015],
       [0.21651876]], dtype=float32)

In [70]:
modelo.evaluate(XwaterPump_test_scaled, ywaterPump_test_nf)

372/372 [==============================] - 2s 4ms/step - loss: 0.5088 - binary_accuracy: 0.7637


[0.5088086724281311, 0.7637205123901367]

In [71]:
features_names = XwaterPump_train.columns

capa_fadl = modelo.model.get_layer('selection_layer')
mask = binary_sigmoid_unit(capa_fadl.get_mask()).numpy()
print(capa_fadl.get_mask())
print(mask)

selected_features = features_names[ mask.flatten().astype(bool).tolist() ].tolist()
selected_features


tf.Tensor([0. 0. 0. ... 0. 0. 1.], shape=(9214,), dtype=float32)
[0. 0. 0. ... 0. 0. 1.]


['extraction_type_gravity',
 'extraction_type_other',
 'extraction_type_group_nira/tanira',
 'management_private operator',
 'payment_type_annually',
 'payment_type_monthly',
 'payment_type_per bucket',
 'quantity_group_dry',
 'quantity_group_enough',
 'waterpoint_type_group_other']

In [72]:
modelo.model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 9214)]            0         
                                                                 
 selection_layer (TimeSelec  (None, 9214)              9214      
 tionLayer)                                                      
                                                                 
 dense1_layer (Dense)        (None, 32)                294880    
                                                                 
 dense2_layer (Dense)        (None, 8)                 264       
                                                                 
 output_layer (Dense)        (None, 1)                 9         
                                                                 
Total params: 304367 (1.16 MB)
Trainable params: 304367 (1.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________